In [ ]:
import ee

In [ ]:
ee.Authenticate(force=True)
ee.Initialize()

In [ ]:
def read_wkt(file_path):
    # Step 1: Read the WKT string from the file
    with open(file_path, 'r') as file:
        wkt_string = file.read()
    return wkt_string


In [ ]:
projection_type = 'Lambert_Azimuthal_Equal_Area'
projection_scale = 30
projection_wkt_path = './projection_wkt/{projection_type}/{hybas_id}.wkt'

hybas_gt100_id_list = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroBASINS_lev02/hybas_lev02_v1c_merged_lake_count_added_gt100").aggregate_array("HYBAS_ID").getInfo()
for hybas_id in hybas_gt100_id_list:
    current_projection_wkt_path = projection_wkt_path.format(projection_type=projection_type, projection_scale=projection_scale, hybas_id=hybas_id)
    current_projection_wkt = read_wkt(current_projection_wkt_path)
    current_region = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroLAKES_per_basin_dissolved/hylak_buffered_bounds_dissolved_{}".format(hybas_id)).geometry(1).bounds(1)
    gsw_max_extent = ee.Image('JRC/GSW1_4/GlobalSurfaceWater').select(['occurrence', 'recurrence'], ['GSW_occurrence', 'GSW_recurrence']).reproject(crs=ee.Projection(current_projection_wkt), scale=projection_scale).unmask(0)
    task = ee.batch.Export.image.toDrive(
        image = gsw_max_extent,
        description = "gsw_occurrence_and_recurrence_30m_laea_{}".format(hybas_id),
        folder = 'gsw_occurrence_and_recurrence',
        region = current_region,
        scale = projection_scale,
        crs = ee.Projection(current_projection_wkt),
        maxPixels = 1e13
    )
    task.start()
    print("Exporting gsw max_extent image for basin {}".format(hybas_id))